In [ ]:
%load_ext nb_black
# Load the "autoreload" extension so that code can change
%load_ext autoreload
# always reload modules so that as you change code in src, it gets loaded
%autoreload 2

In [ ]:
import numpy as np
import os
import pandas as pd
import shap  # package used to calculate Shap values
from matplotlib import pyplot as plt
from rfpimp import *
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import _VectorizerMixin
from sklearn.feature_selection._base import SelectorMixin
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    FunctionTransformer,
    OneHotEncoder,
    OrdinalEncoder,
)
from sklearn.preprocessing._encoders import _BaseEncoder


# from category_encoders import OrdinalEncoder

## NOTE
This notebook code has been written in a mostly procedural format.  This is so that I can demonstrate my thought process and decision making in a clear and logical way.  This code would need some re-factoring once I'm happy with my final solution, before thinking about getting it to a production ready format if required.

I have used 'Black' code reformatter to ensure the style is PEP 8 compliant.

In [ ]:
%load_ext nb_black

In [ ]:


# from category_encoders import OrdinalEncoder


In [ ]:
os.getcwd()

## Feature Engineering
This step of the process is crucial for obtaining an accurate model.  It is also the most time consuming.  I've spent my time creating an end-to-end solution and therefore haven't dived as deeply into all of the necessary steps to produce the best model.
Below I have listed most of the techniques I would normally try:
 - Imputation
- Drop records or columns with a high proportion of missing values
- Replace missing values with mean/median/mode (numeric) or mode (categorical)
 - One-hot Encoding
- Map categorical feature to N-1 binary columns
- Try to not create a sparse dataframe if feature contains many categories
 - Outliers
- Remove outliers entirely
- Capping the outliers using percentile cut-offs
- Binning the values
 - Simplify relationships to either binary flags or straight lines
 - Transformations
- Apply different transformations based on the shape of the distribution
- Square root, logarithm, inverse and reflect the distribution when it is negatively skewed
- Try to straighten the curve of a line to extract a stronger relationship
 - Mean Enconding for categorical features
- For categorical features, encode each level as the mean of the response
- Achieves many objectives including scaling, outliers, NULL values and interpretability
- Binning
- For continuous features try quartiles, quintiles, deciles
 - Binning with Decision Trees
 - Polynomial Features, e.g. xsquared, xcubed
 - Feature Interactions, e.g. x1x2
 - Derived Features using domain knowledge

NOTE:
Random Forest is a tree-based model that uses a partitioning algorithm and does not require feature scaling.  The decision on where to split/partition the data will not change with scaling, i.e. the tree only sees ranks in the features.

In [ ]:
# access store variables
%store -r df
%store -r predict

In [ ]:
# we can do some of the pre-processing on the training and prediction data at the same time
df_combined = pd.concat([df, predict], axis=0, sort=False)
print(df_combined.shape)
df_combined.head(-5)

In [ ]:
# convert object to category dtypes
df_combined["transport_availability"] = df_combined["transport_availability"].astype(
    "category"
)
df_combined["county"] = df_combined["county"].astype("category")

In [ ]:
# map the ordinal values to integers
mapping_dict = {
    "transport_availability": {
        "All transport options": 5,
        "Many transport options": 4,
        "Average transport options": 3,
        "Few transport options": 2,
        "No transport options": 1,
    }
}
df_combined.replace(mapping_dict, inplace=True)
df_combined.head(5)

In [ ]:
# one-hot encode new_store feature
df_combined = pd.get_dummies(df_combined, columns=["new_store"])

# check distribution of feature
print(df_combined["new_store_yes"].value_counts())

df_combined.head(5)

In [ ]:
# encode county values with numeric labels
df_combined["county"] = df_combined["county"].cat.codes
df_combined.head(5)

In [ ]:
# define list of features before further transformed features are added
# orig_features = df_combined.drop(columns=["normalised_sales", "data"]).columns
orig_features = df_combined.drop(columns=["normalised_sales", "data"]).columns
orig_features

In [ ]:
# apply transformations to some of the features

# apply log transformation adding one, to avoid negative values
df_combined["log_proportion_newbuilds"] = df_combined["proportion_newbuilds"].apply(
    np.log1p
)
df_combined["log_public_transport_dist"] = df_combined["public_transport_dist"].apply(
    np.log1p
)
# reflect the distribution first before applying log, since distribution is negatively skewed
df_combined["log_competitor_density"] = (
    (df_combined["competitor_density"].max() + 1) -
    df_combined["competitor_density"]
).apply(np.log1p)

# apply inverse transformation
df_combined["inv_commercial_property"] = df_combined["commercial_property"].apply(
    np.reciprocal
)
# reflect the distribution first before applying inverse, since distribution is negatively skewed
df_combined["inv_competitor_density"] = (
    (df_combined["competitor_density"].max() + 1) -
    df_combined["competitor_density"]
).apply(np.reciprocal)

## Splitting the Dataset
- We should re-use the parameters for the transformations applied to the training set for use on the test set to avoid information leakage
- Any predictions made should also use the same parameters
- If the dataset is sufficiently large we shouldn't expect to see much difference between the train and test parameters since we assume all samples have been drawn from the same distribution
- Information leakage can cause optimism bias in the model evaluation
- We should not use the response in the test set for anything except comparing to our predicted values

In [ ]:
pd.options.mode.chained_assignment = None  # default='warn'

# split data for prediction
predict = df_combined.loc[df_combined["data"] == "predict"]
predict = predict.drop(columns=["normalised_sales", "data"])
print("Predict Features Shape:", predict.shape)

# use a seed value throughout notebook to ensure results are reproducible
seed = 42

# split the target value
X = df_combined.loc[df_combined["data"] == "train"].drop(
    columns=["normalised_sales", "data"]
)
y = df_combined.loc[df_combined["data"] == "train"]["normalised_sales"]

# split the data into training and validation sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=seed
)

print("Training Features Shape:", X_train.shape)
print("Training Labels Shape:", y_train.shape)
print("Validation Features Shape:", X_test.shape)
print("Validation Labels Shape:", y_test.shape)

Scikit-learn estimators assume all values are numerical and that all have and hold meaning.

More advanced techniques that could be tried include:
- To avoid discarding entire rows and/or columns containing missing values we can impute missing values.
- Multivariate Feature Imputation -
 - Model each feature with missing values as a function of other features and use that estimate for imputation
 - It does so in an iterated round-robin fashion: at each step, a feature column is designated as output y and the other feature columns are treated as inputs X.
 - A regressor is fit on (X, y) for known y. Then, the regressor is used to predict the missing values of y.
- KNN Feature Imputation -
 - A euclidean distance metric that supports missing values, nan_euclidean_distances, is used to find the nearest neighbors.
 - Each missing feature is imputed using values from n_neighbors nearest neighbors that have a value for the feature.
 - The feature of the neighbors are averaged uniformly or weighted by distance to each neighbor.
 - If a sample has more than one feature missing, then the neighbors for that sample can be different depending on the particular feature being imputed.

In [ ]:
def replace_missing_values(df):
    df["commercial_property"].fillna(
        (X_train["commercial_property"].median()), inplace=True
    )
    df["inv_commercial_property"].fillna(
        (X_train["inv_commercial_property"].median()), inplace=True
    )
    #     df["log_crime_rate"].fillna(
    #         (X_train["log_commercial_property"].median()), inplace=True
    #     )
    #     df["log_proportion_flats"].fillna(
    #         (X_train["log_proportion_flats"].median()), inplace=True
    #     )
    #     df["inv_crime_rate"].fillna(
    #         (X_train["inv_commercial_property"].median()), inplace=True
    #     )
    #     df["cap_commercial_property"].fillna(
    #         (X_train["cap_commercial_property"].median()), inplace=True
    #     )
    #     df["cap_proportion_flats"].fillna(
    #         (X_train["cap_proportion_flats"].median()), inplace=True
    #     )
    df["school_proximity"].fillna(
        (X_train["school_proximity"].median()), inplace=True)
    return df


replace_missing_values(X_train)
replace_missing_values(X_test)
replace_missing_values(predict)

# check for missing values again
print("X_train missing values:", X_train.isnull().values.any())
print("X_test missing values:", X_test.isnull().values.any())
print("Predict missing values:", predict.isnull().values.any())

In [ ]:
# create histogram showing distribution of commercial_property before capping outliers
hist1 = X_train["commercial_property"].plot(
    kind="hist", bins=20, figsize=(10, 5), title="commercial_property: Not Capped"
)
plt.show()

# cap outliers at the 1% and 99% percentile level


def cap_outliers(df, cap1, cap2):

    commercial_property_percentiles = (
        X_train["commercial_property"].quantile([cap1, cap2]).values
    )
    proportion_flats_percentiles = (
        X_train["proportion_flats"].quantile([cap1, cap2]).values
    )

    np.clip(
        df["commercial_property"],
        commercial_property_percentiles[0],
        commercial_property_percentiles[1],
        inplace=True,
    )
    np.clip(
        df["proportion_flats"],
        proportion_flats_percentiles[0],
        proportion_flats_percentiles[1],
        inplace=True,
    )

    # df['capped_commercial_property'] = np.clip(df['commercial_property'], commercial_property_percentiles[0], commercial_property_percentiles[1])
    # df['capped_proportion_flats'] = np.clip(df['proportion_flats'], proportion_flats_percentiles[0], proportion_flats_percentiles[1])

    return df


cap_outliers(X_train, 0.01, 0.99)
cap_outliers(X_test, 0.01, 0.99)
cap_outliers(predict, 0.01, 0.99)

# create histogram showing distribution of commercial_property after capping outliers
hist2 = X_train["commercial_property"].plot(
    kind="hist", bins=20, figsize=(10, 5), title="commercial_property: Capped"
)
plt.show()

In [ ]:
# define labels for bins
labels = ["low", "high"]

# bin using quartile cut-off to ensure even split of samples
X_train["binned_commercial_property"], bins_commercial_property = pd.qcut(
    X_train["commercial_property"], q=2, retbins=True, labels=labels
)
# bin using cut since distribution of this feature is so imbalanced
X_train["binned_proportion_flats"], bins_proportion_flats = pd.cut(
    X_train["proportion_flats"], 2, retbins=True, labels=labels
)

# apply bin cut-offs from training data to test set
X_test["binned_commercial_property"] = pd.cut(
    X_test["commercial_property"], bins=bins_commercial_property, labels=labels
)
X_test["binned_proportion_flats"] = pd.cut(
    X_test["proportion_flats"], bins=bins_proportion_flats, labels=labels
)

# apply bin cut-offs from training data to predict set
predict["binned_commercial_property"] = pd.cut(
    predict["commercial_property"], bins=bins_commercial_property, labels=labels
)
predict["binned_proportion_flats"] = pd.cut(
    predict["proportion_flats"], bins=bins_proportion_flats, labels=labels
)

# one-hot encode new binned features
X_train = pd.get_dummies(
    X_train, columns=["binned_commercial_property", "binned_proportion_flats"]
)
X_test = pd.get_dummies(
    X_test, columns=["binned_commercial_property", "binned_proportion_flats"]
)
predict = pd.get_dummies(
    predict, columns=["binned_commercial_property", "binned_proportion_flats"]
)

X_train.head(5)

# Pipelines
To simplify the process of applying transformations it's best to use a pipeline. They have several key benefits:
 - They make the workflow much easier to read and understand.
 - They enforce the implementation and order of steps in the project.
 - These in turn make the work much more reproducible.

If we receive further prediction data in future or this model needs to be put into production then it's much more desirable to simply pass the data through a pipeline.

NOTE - Due to time constraints I haven't fully constructed a pipeline with all of the necessary components.

In [ ]:
# this implementation of OrdinalEncoder is required to allow for unknown categories in unseen data
# the handle_categories option has not been rleased in sklearn yet
# Do not use from sklearn.preprocessing import _BaseEncoder, it is protected class!


class new_OrdinalEncoder(_BaseEncoder):
    def __init__(self, cat_index="all"):
        self.dicts = {}
        # cate_index is the categorical feature index list
        self.cat_index = cat_index

    def fit(self, df, *y):
        if self.cat_index == "all":
            self.cat_index = list(range(df.shape[1]))
        for feat in self.cat_index:
            dic = np.unique(df.iloc[:, feat])
            dic = dict([(i, index) for index, i in enumerate(dic)])
            self.dicts[feat] = dic

    def fit_transform(self, df, *y):
        if self.cat_index == "all":
            self.cat_index = list(range(df.shape[1]))
        df_output = df.copy()
        for feat in self.cat_index:
            dic = np.unique(df.iloc[:, feat])
            dic = dict([(i, index) for index, i in enumerate(dic)])
            self.dicts[feat] = dic
            df_output.iloc[:, feat] = df.iloc[:, feat].apply(lambda x: dic[x])
        return df_output

    def transform(self, df):
        df_output = df.copy()
        for feat in self.cat_index:
            dic = self.dicts[feat]
            df_output.iloc[:, feat] = df.iloc[:, feat].apply(
                self.unknown_value, args=(dic,)
            )
        return df_output

    def unknown_value(
        self, value, dic
    ):  # It will set up a new interger for unknown values!
        try:
            return dic[value]
        except:
            return len(dic)

OrdinalEncoder
 - If a new value that never appears in the fit step needs to be converted in the transform step, OrdinalEncoder will raise an error.
 - Such unknown value will cause a problem when the testing set contains an unknown value of the training set.
 - In order to avoid unknown value in the testing set, we have to fit the entire data set for the OrdinalEncoder, which means we need to fit the OrdinalEncoder before splitting the dataset into training and testing.
 - Even if we can fix unknown value in this way, it still does not work when new samples with unknown value fed into the model.
 - Due to the above I have used a custom implementation of OrdinalEncoder that handles missing values.

In [ ]:
# df = train.copy()

# define the ordering for the categories used in the mapping of the ordinal feature
categories = [
    "All transport options",
    "Many transport options",
    "Average transport options",
    "Few transport options",
    "No transport options",
]

# replace missing values with median
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median"))])

# apply logarithm transform
log_transformer = Pipeline(steps=[("log", FunctionTransformer(np.log1p))])

# apply inverse transform
inv_transformer = Pipeline(steps=[("inv", FunctionTransformer(np.reciprocal))])

# one-hot encode categorical feature
# if an unknown category is encountered during transform, the resulting ohe columns for this feature will all be zeros
categorical_transformer1 = Pipeline(
    steps=[("onehot", OneHotEncoder(handle_unknown="ignore"))]
)

# transform categorical feature to numeric with ordinal mapping
categorical_transformer2 = Pipeline(
    steps=[("ordinal1", OrdinalEncoder(categories=[categories]))]
)

# transform categorical feature to numeric with lexical (default method) ordering
# use a different implementation of OrdinalEncoder above to allow for unknown categories not seen in training data
categorical_transformer3 = Pipeline(steps=[("ordinal2", new_OrdinalEncoder())])

numeric_features = (
    train.drop(columns=["normalised_sales"])
    .select_dtypes(include=["int64", "float64"])
    .columns
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features)  # ,
        #        ('log', log_transformer, ['proportion_newbuilds','public_transport_dist'])
        ,
        #        ('inv', inv_transformer, ['commercial_property'])
        #        ,
        ("cat1", categorical_transformer1, ["new_store"]),
        ("cat2", categorical_transformer2, ["transport_availability"]),
        ("cat3", categorical_transformer3, ["county"]),
    ]
)

In [ ]:
# check that transformations are being applied correctly
def get_feature_out(estimator, feature_in):
    if hasattr(estimator, "get_feature_names"):
        if isinstance(estimator, _VectorizerMixin):
            # handling all vectorizers
            return [f"vec_{f}" for f in estimator.get_feature_names()]
        else:
            return estimator.get_feature_names(feature_in)
    elif isinstance(estimator, SelectorMixin):
        return np.array(feature_in)[estimator.get_support()]
    else:
        return feature_in


def get_ct_feature_names(ct):
    # handles all estimators, pipelines inside ColumnTransfomer
    # doesn't work when remainder =='passthrough'
    # which requires the input column names.
    output_features = []

    for name, estimator, features in ct.transformers_:
        if name != "remainder":
            if isinstance(estimator, Pipeline):
                current_features = features
                for step in estimator:
                    current_features = get_feature_out(step, current_features)
                features_out = current_features
            else:
                features_out = get_feature_out(estimator, features)
            output_features.extend(features_out)
        elif estimator == "passthrough":
            output_features.extend(ct._feature_names_in[features])

    return output_features


transformed_data = preprocessor.fit_transform(train)

pd.DataFrame(transformed_data, columns=get_ct_feature_names(preprocessor))